# Trouver et appliquer les règles à entropie nulle


## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [3]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/StemSpaces/LongitudinalesRnd/"
fichiers=glob.glob(rep+"*X-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
samples={int(s.split("-")[1]):s for s in samples}
samples

{1: 'Longitudinal-01-T1000000-F29754-X-',
 2: 'Longitudinal-02-T1000000-F29754-X-'}

In [4]:
sample=samples[1]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [5]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [6]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

In [7]:
numero=1
typeRules="PMS"
avecContexte=True

In [8]:
rules=openRules(numero,typeRules=typeRules)

In [9]:
def getPatronsConsideres(rules,avecContexte=False):
    patronsConsideres={}
    patronsAvecContexte={}
    patronsSansContexte={}
    for paire in rules.keys():
        patronsAvecContexte[paire]=rules[paire].patrons
        for r in patronsAvecContexte[paire]:
            if paire not in patronsSansContexte:
                patronsSansContexte[paire]={}
            transformation=r.split("-")[0]
            patronsSansContexte[paire][r]=u"^(.*)%s$"%transformation
    if not avecContexte:
        patronsConsideres=patronsSansContexte
    else:
        patronsConsideres=patronsAvecContexte
    return patronsConsideres

In [10]:
rules[("ii1S","ii2P")].patrons

{u'E-e': u'^(.*[ptkbdgfsSvzZmnJjlrE6a\xea\xe2][ptkbdgfsSvzZmnJjrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZjrwH][iEea])jE$',
 u'E-ije': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZ][lr])E$',
 u'E-je': u'^(.*[ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJjlrE6\xea])E$',
 u'jE-ije': u'^(.*[bdvzmnjiEe\xea][ptkbdgjwHiyEe926auOo][ptbdfsvzmnr][iyEe926uOo][fsvzl])jE$',
 u'wE-je': u'^\xe2buwE$'}

In [11]:
zeroEntropie=[]
for paire in rules:
    classe=rules[paire].classe
    c1,c2=paire
    if c1!=c2:
        zero=True
        print paire
        for transformation in classe:
            choix=classe[transformation]
            if len(choix)>1:
                print choix
                zero=False
        if zero:
            print "zéro",choix
            zeroEntropie.append(paire)
        print

(u'ps2P', u'is1S')
zéro {u'je-was': 1}

(u'ii3S', u'ps3P')
zéro {u'EE-e': 1}

(u'pc2S', u'ai2P')
zéro {u'6rE-at': 2}

(u'is2S', u'pc3P')
zéro {u'6rE-at': 2}

(u'ps2S', u'ii2P')
zéro {u'His-uvje': 1}

(u'pc2P', u'ai2P')
zéro {u'E.6rje-6.at': 1}

(u'ii3P', u'pi2P')
zéro {u'E-e': 4063}

(u'fi3P', u'pi2P')
{u'ir\xf4-je': 60, u'r\xf4-se': 45}
{u'6r\xf4-e': 78, u'E.6r\xf4-6.e': 47}

(u'pi2S', u'ps3P')
zéro {'-': 965}

(u'pc1P', u'pi2S')
zéro {u'Orj\xf4-o': 1}

(u'is2P', u'pP')
zéro {u'Orj\xf4-o': 1}

(u'is2S', u'ai1S')
zéro {u'as-E': 1}

(u'fi1P', u'pI1P')
zéro {u'j\xeadr\xf4-6n\xf4': 1}

(u'ps2P', u'ai3S')
zéro {u'je-i': 1}

(u'ppFP', u'pi2S')
{u'e-': 1, u'je-': 1}
{u'e-': 26, u'O.e-o.': 55}

(u'ii3S', u'ii3P')
zéro {'-': 4217}

(u'fi2S', u'ii1P')
{u'6ra-j\xf4': 697, u'ra-zj\xf4': 1}

(u'pP', u'pi1P')
zéro {u'\xe2-\xf4': 3814}

(u'ps2S', u'fi1S')
{u'-rE': 6, u'-6rE': 162}

(u'pi2P', u'fi1S')
{u'je-rE': 1, u'e-6rE': 1}
{u'ze-rE': 2, u'e-6rE': 134}
{u'e-rE': 45, u'e-6rE': 1181}
{u'se-rE': 21,

(u'fi1P', u'pc1P')
zéro {u'\xf4-j\xf4': 1531}

(u'ii3S', u'pI2S')
{u'E-': 66, u'O.E-o.': 46}
{u'E-': 137, u'zE-': 13}
{u'E-': 31, u'jE-i': 95}

(u'ii2P', u'is1P')
zéro {u'je-asj\xf4': 1}

(u'ps3S', u'inf')
{u'-e': 29, u'E.-6.e': 28}
{u'-je': 7, u'-r': 2}
{u'-e': 873, u'-r': 45}
{u'-e': 139, u'z-r': 7}
{u'-e': 19, u'E.-6.e': 22, u'-r': 3}
{u'-e': 7, u's-r': 140}
{u'-e': 103, u'-ir': 3, u'-r': 4}
{u'-e': 49, u'-ir': 2}

(u'is2P', u'fi1P')
zéro {u'9.-6.war': 1}

(u'fi2P', u'pi1S')
{u'6re-': 89, u'O.6re-o.': 52}
{u'6re-': 148, u'O.6re-o.': 1}
{u'j6re-': 1, u'6re-': 1}

(u'ps1P', u'is1S')
zéro {u'j\xf4-as': 4}

(u'is2S', u'ps3S')
zéro {u'as-': 1}

(u'pI1P', u'pc1S')
{u'\xf4-rE': 14, u'\xf4-6rE': 1138}

(u'ii2S', u'ai1S')
zéro {'-': 1144}

(u'fi3P', u'pc1S')
zéro {u'\xf4-E': 3126}

(u'ps1S', u'ai2S')
zéro {u'o.-O.a': 1}

(u'pi2S', u'pc2S')
zéro {u'-rE': 7}

(u'ps1S', u'pi1P')
zéro {u'o.-O.\xf4': 1}

(u'fi3P', u'ai2P')
zéro {u'E.6r\xf4-6.at': 1}

(u'is3S', u'pI2S')
zéro {u'ly-': 1}

(u'ai2P',

{u'6.\xe2-E.': 1, u'6n\xe2-j\xea': 4}
{u'\xe2-': 81, u't\xe2-': 12}
{u'\xe2-': 2, u'j\xe2-': 24}
{u'EJ\xe2-\xea': 13, u'\xe2-': 5}
{u'6.\xe2-E.': 32, u'6.\xe2-9.': 2}
{u'\xe2-': 48, u'j\xe2-i': 157}
{u'\xe2-': 155, u'z\xe2-': 21}
{u'\xe2-': 369, u'O.\xe2-o.': 58}
{u'\xe2-': 4, u'j\xe2-': 3}
{u'\xe2-': 13, u'd\xe2-': 25}

(u'pI2S', u'fi2P')
{u'-dre': 3, u'-6re': 172}

(u'pc1P', u'ai1S')
zéro {u'E.6rj\xf4-6.E': 1}

(u'ii2P', u'inf')
{u'je-r': 5, '-': 5}
{u'je-r': 14, u'je-e': 177}
{'-': 1, u'ije-e': 44}

(u'ppMS', u'pi3P')
{u'-s': 122, u'-z': 14}
{u'e-': 320, u'O.e-o.': 5}
{u'e-': 27, u'je-i': 34}
{u'6.e-9.': 3, u'6.e-E.': 27}
{u'e-': 154, u'O.e-o.': 64}
{u'e-': 12, u'je-': 41}
{u'e-': 6, u'je-': 3}
{'-': 3, u'-s': 5, u'-z': 2}
{u'e-': 5, u'je-i': 115}
{u'i-': 7, u'-s': 11}
{u'6.e-9.': 1, u'6.e-E.': 1}

(u'ai2P', u'pc1P')
zéro {u'at-6rj\xf4': 4}

(u'fi2S', u'ps1P')
zéro {u'ra-j\xf4': 1}

(u'ai1P', u'ii2S')
zéro {u'am-E': 893}

(u'ai3P', u'is1S')
zéro {u'Er-as': 75}

(u'is2S', u'fi3P')
zé

{u'E-': 13, u'dE-': 26}
{u'E-': 10, u'O.E-o.': 14}
{u'E-': 7, u'jE-': 41}
{u'E-': 53, u'sE-': 225}
{u'E-': 47, u'jE-i': 163}
{u'E-': 1, u'lE-': 1}

(u'pc3P', u'ii3S')
{u'E.6rE-6.E': 36, u'6rE-E': 12}
{u'r.-z.': 1, u'irE-jE': 13}
{u'irE-jE': 18, u'r.-s.': 3}

(u'ps2P', u'is1P')
zéro {u'je-asj\xf4': 1}

(u'ppFP', u'ii3S')
{u'y-izE': 2, u'y-E': 1}

(u'ps3P', u'ai2S')
zéro {u'i-ja': 1}

(u'pP', u'pc1P')
zéro {u'\xe2-6rj\xf4': 1685}

(u'is2S', u'pI1P')
zéro {u'as-\xf4': 1}

(u'pP', u'fi2S')
{u'\xe2-6ra': 11, u'j\xe2-ira': 69}
{u'\xe2-6ra': 579, u'\xe2-ra': 39}
{u'\xe2-6ra': 256, u'z\xe2-ra': 13}

(u'pc2P', u'pi1P')
zéro {u'irje-\xf4': 1}

(u'pc2P', u'is3P')
zéro {u'rje-was': 1}

(u'pP', u'pc3S')
{u's\xe2-rE': 66, u'\xe2-6rE': 2}
{u'z\xe2-rE': 17, u'\xe2-6rE': 147}
{u'\xe2-rE': 4, u'\xe2-6rE': 252, u'\xe2-irE': 8}
{u'\xe2-rE': 48, u'\xe2-6rE': 2136}
{u'\xe2-6rE': 2, u'v\xe2-rE': 2}
{u'j\xe2-rE': 2, u'\xe2-6rE': 4}
{u'\xe2-6rE': 1, u'j\xe2-irE': 125}

(u'pI1P', u'fi1S')
{u'\xf4-rE': 16, u'\xf

{u'j\xf4-': 5, u'sj\xf4-': 7}
{u'j\xf4-': 2, u'ij\xf4-': 20}
{u'j\xf4-': 14, u'O.j\xf4-o.': 56}
{u'j\xf4-': 44, u'\xf4-': 8}
{u'j\xf4-': 78, u'tj\xf4-': 6}

(u'ii2S', u'ps2P')
zéro {u'wE-je': 1}

(u'ppMS', u'ps3S')
{u'i-': 6, u'-s': 6}
{u'-s': 2, u'-z': 13}
{u'y-': 1, u'y-iz': 2}
{u'e-': 161, u'O.e-o.': 51}

(u'pI2S', u'ai1S')
{u'-E': 80, u'-ti': 1}
{u'i-y': 2, u'i-jE': 3}
{u'-E': 37, u'E.-6.E': 11}

(u'pc2S', u'fi2P')
zéro {u'E-e': 2077}

(u'pc1S', u'pP')
{u'rE-\xe2': 4, u'rE-z\xe2': 1}
{u'irE-j\xe2': 34, u'rE-s\xe2': 9}
{u'6rE-\xe2': 1492, u'rE-z\xe2': 1}
{u'6rE-\xe2': 13, u'E.6rE-6.\xe2': 5}
{u'irE-j\xe2': 56, u'rE-z\xe2': 1}

(u'ai1P', u'pi3S')
{u'O.am-o.': 24, u'am-': 10}

(u'pI2P', u'is1P')
zéro {u'e-asj\xf4': 1}

(u'is1S', u'ppMP')
zéro {u'as-e': 113}

(u'inf', u'pI2P')
{u'r-ze': 2, u'r-se': 1}

(u'ppFS', u'ai1P')
zéro {u'e-am': 2668}

(u'fi3S', u'ii2P')
{u'6ra-je': 1, u'6ra-ije': 56}

(u'ai1P', u'pc1S')
zéro {u'lym-drE': 1}

(u'fi3P', u'pP')
{u'r\xf4-s\xe2': 25, u'ir\xf4-j\xe2'

In [12]:
zeroPop={}
for paire in zeroEntropie:
    for t in rules[paire].classe:
        pop=0
        for c in rules[paire].classe[t]:
            pop+=rules[paire].classe[t][c]
        zeroPop[paire]=pop
    

In [43]:
stemspaceThresh=700
stemspaceRules=[]
for k in sorted(zeroPop, key=zeroPop.get,reverse=True):
    print k, zeroPop[k]
    if zeroPop[k]>=stemspaceThresh:
        stemspaceRules.append(k)

(u'ii3S', u'ii3P') 4217
(u'ii3P', u'ii3S') 4217
(u'pP', u'ii3S') 4185
(u'ii1S', u'ii3S') 4121
(u'ii3S', u'ii1S') 4121
(u'pP', u'ii3P') 4085
(u'ii3P', u'pi2P') 4063
(u'pi2P', u'ii3P') 4063
(u'ii1S', u'ii3P') 4050
(u'ii3P', u'ii1S') 4050
(u'pP', u'ii1S') 4005
(u'ppFS', u'ppFP') 3856
(u'pi2P', u'pI2P') 3824
(u'pI2P', u'pi2P') 3824
(u'pP', u'pi1P') 3814
(u'pi1P', u'pP') 3814
(u'ii3P', u'pI2P') 3802
(u'pI2P', u'ii3P') 3802
(u'pP', u'pI2P') 3740
(u'pc3S', u'pc1S') 3552
(u'pc1S', u'pc3S') 3552
(u'pc1S', u'fi3S') 3536
(u'fi3S', u'pc1S') 3536
(u'ppMS', u'ai3P') 3506
(u'pc1S', u'fi1S') 3503
(u'fi1S', u'pc1S') 3503
(u'ppMP', u'ai3P') 3477
(u'ppFS', u'ai3P') 3471
(u'is3S', u'ai3S') 3464
(u'ai3S', u'is3S') 3464
(u'ai3P', u'ai1S') 3420
(u'ai1S', u'ai3P') 3420
(u'ppFP', u'ai3P') 3365
(u'is3S', u'pi1P') 3350
(u'fi3P', u'fi3S') 3271
(u'fi3S', u'fi3P') 3271
(u'fi3P', u'pc3S') 3232
(u'fi3P', u'fi1S') 3187
(u'fi1S', u'fi3P') 3187
(u'pc3S', u'fi2S') 3178
(u'fi1S', u'fi2S') 3175
(u'pc1S', u'fi2S') 3151
(u'f

(u'ai2P', u'pi1P') 1
(u'inf', u'pc1P') 1
(u'is2P', u'ppFS') 1
(u'pc2S', u'ai3S') 1
(u'is2S', u'fi2S') 1
(u'pP', u'ai1P') 1
(u'ppFP', u'ps3P') 1
(u'pc2S', u'is1S') 1
(u'ii3P', u'ai2S') 1
(u'ppMS', u'ps1P') 1
(u'ps3P', u'ai3S') 1
(u'is1S', u'pc3S') 1
(u'ppMP', u'ii2S') 1
(u'ai2S', u'pi3P') 1
(u'ps1S', u'ps2P') 1
(u'is2S', u'pc2S') 1
(u'is3P', u'pc3S') 1
(u'ps1P', u'fi1P') 1
(u'ii1S', u'ai1P') 1
(u'ii2S', u'pc2P') 1
(u'pc2P', u'ppFS') 1
(u'pI1P', u'is3P') 1
(u'ps1S', u'ii2S') 1
(u'is2S', u'pI2S') 1
(u'pi3P', u'is3P') 1
(u'ai3S', u'ii2S') 1
(u'ps1S', u'fi3P') 1
(u'ps3P', u'inf') 1
(u'pc1S', u'is3P') 1
(u'ai2S', u'pc3P') 1
(u'ps1S', u'pc1S') 1
(u'ppMS', u'pI1P') 1
(u'fi2S', u'ai1P') 1
(u'is2S', u'ii3P') 1
(u'ai3P', u'ps3P') 1
(u'ai1P', u'pi1S') 1
(u'pc3P', u'is1P') 1
(u'ppFS', u'pI1P') 1
(u'ppFS', u'ps1S') 1
(u'is1P', u'ps3S') 1
(u'ai2S', u'ii3P') 1
(u'is1S', u'fi2S') 1
(u'ii2P', u'ai2S') 1
(u'ai2S', u'ps3P') 1
(u'pi3P', u'is2S') 1
(u'pc1P', u'pc1S') 1
(u'ii1S', u'fi1P') 1
(u'ai2S', u'ps2P'

In [44]:
len(stemspaceRules)

218

In [28]:
fSamplePMS="paradigmes.csv"
fSamplePMO="Morphomes-paradigmes.csv"

In [29]:
def openSample(numero,typeSample="PMS"):
    result=None
    fSample=""
    if typeSample=="PMS" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMS
    elif typeSample=="PMO" and numero in samples:
        fSample=rep+samples[numero]+fSamplePMO
    if fSample:
        result=pd.read_csv(rep+samples[numero]+fSamplePMS,sep=";",index_col=0,keep_default_na=False,encoding="utf8")
    return result    

In [30]:
df=openSample(1,"PMS")

In [31]:
def makeZeros(sampleFormes,inFormes=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=rules.keys()
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c])
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [32]:
def fillStemSpace(sampleFormes,inFormes=True):
    series=sampleFormes.copy()
    series={k:v for k,v in series.iteritems() if v!=""}

    paires=stemspaceRules
    formesC={}
    for c in series:
        if not c in formesC:
            formesC[c]=set()
        formesC[c].add(series[c])
        if debug: print c, series[c]
        formeC=series[c]
        pairesC=[(c1,c2) for (c1,c2) in paires if c1==c]
        for p in pairesC:
            formeP=rules[p].sortirForme(series[c])
            if formeP and len(formeP)==1:
                p1=p[1]
                if debug: print p1,formeP
                if p1 not in formesC:
                    formesC[p1]=set()
                formesC[p1].add(formeP.keys()[0])
    if inFormes:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1}
    else:
        formesZero={k:list(v)[0] for k,v in formesC.iteritems() if len(v)==1 and k not in series}

    return formesZero

In [33]:
cases=[c for c in df.columns.tolist() if c!="lexeme"]
cases=sorted(cases)

In [37]:
lexemes=u"abreuver clore paître distraire être avoir devoir pouvoir pendre perdre prendre".split(" ")
#lexemes=df["lexeme"].tolist()
lexemes

[u'abreuver',
 u'clore',
 u'pa\xeetre',
 u'distraire',
 u'\xeatre',
 u'avoir',
 u'devoir',
 u'pouvoir',
 u'pendre',
 u'perdre',
 u'prendre']

In [38]:
for lexeme in lexemes:
    index=df.loc[df['lexeme']==lexeme].index.values
    if index:
        i=index[0]
        sampleFormes=df.iloc[i].to_dict()
        sampleFormes={k:v for k,v in sampleFormes.iteritems() if v!=""}
        lexemeI=sampleFormes["lexeme"]
        print lexemeI
#        formesZero=makeZeros(sampleFormes,inFormes=False)
        formesZero=fillStemSpace(sampleFormes,inFormes=False)
        for case in cases:
            if case in sampleFormes:
                print "\t",case,"\t  ",sampleFormes[case]
            elif case in formesZero:
                print "\t",case,"\t=>",formesZero[case]
            else:
                print "\t",case
        print

abreuver
	ai1P
	ai1S
	ai2P
	ai2S 	=> abr6va
	ai3P 	=> abr6vEr
	ai3S
	fi1P
	fi1S
	fi2P
	fi2S
	fi3P
	fi3S
	ii1P
	ii1S 	=> abr6vE
	ii2P
	ii2S 	=> abr6vE
	ii3P 	   abr6vE
	ii3S 	   abr6vE
	inf 	   abr6ve
	is1P
	is1S 	=> abr6vas
	is2P
	is2S
	is3P
	is3S
	pI1P
	pI2P 	=> abr6ve
	pI2S
	pP
	pc1P 	=> abr6v6rjô
	pc1S
	pc2P
	pc2S
	pc3P
	pc3S
	pi1P
	pi1S
	pi2P 	=> abr6ve
	pi2S
	pi3P 	   abr9v
	pi3S 	   abr9v
	ppFP
	ppFS
	ppMP
	ppMS 	   abr6ve
	ps1P
	ps1S
	ps2P 	=> abr6vje
	ps2S
	ps3P 	=> abr9v
	ps3S

distraire
	ai1P
	ai1S
	ai2P
	ai2S
	ai3P
	ai3S
	fi1P
	fi1S
	fi2P
	fi2S
	fi3P
	fi3S
	ii1P
	ii1S
	ii2P
	ii2S
	ii3P
	ii3S
	inf 	   distrEr
	is1P
	is1S
	is2P
	is2S
	is3P
	is3S
	pI1P
	pI2P
	pI2S
	pP
	pc1P
	pc1S
	pc2P
	pc2S
	pc3P
	pc3S
	pi1P
	pi1S
	pi2P
	pi2S
	pi3P 	   distrE
	pi3S 	   distrE
	ppFP 	=> distrEt
	ppFS 	   distrEt
	ppMP 	   distrE
	ppMS 	   distrE
	ps1P
	ps1S
	ps2P
	ps2S
	ps3P 	=> distrE
	ps3S

être
	ai1P
	ai1S
	ai2P
	ai2S
	ai3P
	ai3S
	fi1P
	fi1S
	fi2P 	   s6re
	fi2S 	=> s6ra
	fi3P
	fi3S
	ii1P
	i

In [ ]:
for i in range(500):
    sampleFormes=df.loc[i].to_dict()
    lexemeI=sampleFormes["lexeme"]
    if not lexemeI.endswith("er"):
        print lexemeI
        numero=1
        for k,v in sampleFormes.iteritems():
            if k!="lexeme" and v!="":
                print numero,k,v
                numero+=1
        print
        formesZero=makeZeros(sampleFormes,inFormes=False)
        for k,v in formesZero.iteritems():
            if k!="lexeme":
                print numero,k,v
                numero+=1
        print

In [ ]:
rules[("pi3S","pi1S")].sortirForme("Xi")